In [1]:
import os
from subprocess import check_call
from glob import glob
from collections import OrderedDict

import numpy as np
import pandas as pd

In [2]:
files = []
dirs = sorted(glob('data/????-????'))

regions = ['ENC', 'WNC']

all_files = []
for r in regions:
    for d in dirs:
        files_here = sorted(
            glob(f'{d}/{r}*.zip') +     glob(f'{d}/{r}*.ZIP')
        )
        all_files.extend(files_here)
all_files        

['data/1985-1987/ENCEN85.zip',
 'data/1985-1987/ENCEN86.zip',
 'data/1985-1987/ENCEN87.zip',
 'data/1988-1990/ENCEN88.zip',
 'data/1988-1990/ENCEN89.zip',
 'data/1988-1990/ENCEN90.zip',
 'data/1991-1993/ENCEN91.zip',
 'data/1991-1993/ENCEN92.zip',
 'data/1991-1993/ENCEN93.zip',
 'data/1994-1996/ENCEN94.zip',
 'data/1994-1996/ENCEN95.zip',
 'data/1994-1996/ENCEN96.zip',
 'data/1994-1996/ENCEN96_wIL.zip',
 'data/1997-1999/ENCEN97.zip',
 'data/1997-1999/ENCEN98.zip',
 'data/1997-1999/ENCEN99.zip',
 'data/2000-2002/ENCEN00.zip',
 'data/2000-2002/ENCEN01.zip',
 'data/2000-2002/ENCEN02.zip',
 'data/2003-2005/ENCEN03.zip',
 'data/2003-2005/ENCEN04.zip',
 'data/2003-2005/ENCEN05.zip',
 'data/2006-2008/ENCEN06.ZIP',
 'data/2006-2008/ENCEN07.ZIP',
 'data/2006-2008/ENCEN08.zip',
 'data/2009-2011/ENCEN09.ZIP',
 'data/2009-2011/ENCEN10.zip',
 'data/2009-2011/ENCEN11.zip',
 'data/2012-2015/ENCEN12.zip',
 'data/2012-2015/ENCEN13.zip',
 'data/2012-2015/ENCEN14.zip',
 'data/2012-2015/ENCEN15.zip',
 'da

In [3]:
def readline(line):
    """parse a single line of fertilizer data file"""
    
    column_defs = OrderedDict([
        ('Fertilizer Year', dict(nchar=2, numeric=True, scale_factor=1)), 
        ('Extra county data', dict(nchar=1, numeric=False)),
        ('State abbr', dict(nchar=2, numeric=False)),
        ('County FIPS code', dict(nchar=3, numeric=True, scale_factor=1)),
        ('Reporting period', dict(nchar=2, numeric=True, scale_factor=1)), 
        ('Quantity (tons)', dict(nchar=9, numeric=True, scale_factor=0.01)),
        ('Fertilizer code', dict(nchar=3, numeric=True, scale_factor=1)),
        ('Container', dict(nchar=1, numeric=True, scale_factor=1)), 
        ('Use', dict(nchar=1, numeric=True, scale_factor=1)), 
        ('Grade: N', dict(nchar=3, numeric=True, scale_factor=0.1)), 
        ('Grade: P', dict(nchar=3, numeric=True, scale_factor=0.1)),
        ('Grade: K', dict(nchar=3, numeric=True, scale_factor=0.1)),
    ])    
    
    ischar = ['Extra county data', 'State abbr']
    
    ndx0 = 0
    data = {}
    for name, info in column_defs.items():
        n = info['nchar']
        value = line[ndx0:ndx0+n]
        if info['numeric']:
            try:
                value = int(value) * info['scale_factor']        
            except:
                value = np.nan       
        data[name] = value
        ndx0 += n
    return data


def file_to_df(file_in):
    """convert dumbass data format to dataframe"""
    with open(file_in, 'r') as fid:
        lines = fid.readlines()
    lines = [l.strip() for l in lines]

    df_lines = []        
    for line in lines:
        df_lines.append(readline(line))
        
    return pd.DataFrame(df_lines)


filter_codes = dict(
    S_codes=[16, 20, 24, 25, 27, 29, 31, 50, 64, 73, 77, 
             207, 263, 265, 267, 443, 463, 
             613, 629, 649, 652, 661, 663, 665, 667, 
             702, 714, 720, 726, 728, 732, 734, 736, 744, 754, 
             770, 774, 780, 782, 783,],
    N_codes=[2, 6, 10, 12, 13, 16, 20, 24, 25, 27, 29, 31, 35, 38, 43,
            46, 50, 52, 54, 56, 58, 59, 60, 61, 62, 64, 66, 67, 68, 73, 
            77, 97, 98, 201, 202, 203, 204, 206, 207, 209, 212, 214, 216, 
            241, 249, 453, 458, 478, 601, 604, 607, 610, 613, 615, 617,
            629, 649, 652, 661, 663, 665, 667, 671, 673, 675, 681, 685,],
    P_codes=[201, 202, 203, 204, 205, 206, 207, 208, 209, 212, 214, 216, 218, 
            223, 228, 233, 238, 241, 243, 248, 249, 253, 263, 265, 267, 273, 
            297, 298, 448, 604, 607, 610, 613, 615, 617, 629, 649, 652, 661, 663,
            665, 667, 671, 673, 681, 685],
    K_codes=[408, 413, 415, 423, 428, 430, 443, 448, 453, 458, 463, 
             478, 497, 498, 604, 607, 610, 613, 615, 617, 629, 649, 652, 661, 
             665, 667, 671, 673, 681, 685]
)


def filter_df(df, filter_names=[]):
    """filter dataframe based on specified values; return dataframe including only matching rows"""

    sel = np.ones(len(df)).astype(np.bool)    
    for filter_name in filter_names:
        sel = sel & df['Fertilizer code'].isin(filter_codes[filter_name])
        
    return df.loc[sel]

In [4]:
diro = 'data/csv_output'
os.makedirs(diro, exist_ok=True)

dfs = {k: [] for k in filter_codes.keys()}
for this_file in all_files:

    if 'ENCEN96_wIL.zip' in this_file:
        print('hardwired', end=',')
        file_in = 'ENCEN96.R'
        file_out = f'{diro}/{file_in}'.replace('.R', '_wIL.csv.gz')
    else:
        file_in = os.path.basename(this_file).replace('.zip', '.R').replace('.ZIP', '.R')        
        file_out = f'{diro}/{file_in}'.replace('.R', '.csv.gz')
    

    if not os.path.exists(file_in):
        check_call(['unzip', this_file])

    
    for filter_name in filter_codes.keys():
        file_out_i = file_out.replace('.csv.gz', f'.{filter_name}.csv.gz')
        print(f'converting: {file_in} --> {file_out_i}')
        df = file_to_df(file_in)
        df = filter_df(df, filter_names=[filter_name])    
        df.to_csv(file_out_i, compression='gzip',)
        dfs[filter_name].append(df)
        
    os.remove(file_in)
    
    
for k, df_list in dfs.items():   
    file_out = f'{diro}/{"-".join(regions)}.{k}.alldata.csv.gz'
    df = pd.concat(df_list)
    df.to_csv(file_out, compression='gzip')
df

converting: ENCEN85.R --> data/csv_output/ENCEN85.S_codes.csv.gz
converting: ENCEN85.R --> data/csv_output/ENCEN85.N_codes.csv.gz
converting: ENCEN85.R --> data/csv_output/ENCEN85.P_codes.csv.gz
converting: ENCEN85.R --> data/csv_output/ENCEN85.K_codes.csv.gz
converting: ENCEN86.R --> data/csv_output/ENCEN86.S_codes.csv.gz
converting: ENCEN86.R --> data/csv_output/ENCEN86.N_codes.csv.gz
converting: ENCEN86.R --> data/csv_output/ENCEN86.P_codes.csv.gz
converting: ENCEN86.R --> data/csv_output/ENCEN86.K_codes.csv.gz
converting: ENCEN87.R --> data/csv_output/ENCEN87.S_codes.csv.gz
converting: ENCEN87.R --> data/csv_output/ENCEN87.N_codes.csv.gz
converting: ENCEN87.R --> data/csv_output/ENCEN87.P_codes.csv.gz
converting: ENCEN87.R --> data/csv_output/ENCEN87.K_codes.csv.gz
converting: ENCEN88.R --> data/csv_output/ENCEN88.S_codes.csv.gz
converting: ENCEN88.R --> data/csv_output/ENCEN88.N_codes.csv.gz
converting: ENCEN88.R --> data/csv_output/ENCEN88.P_codes.csv.gz
converting: ENCEN88.R -->

,Fertilizer Year,Extra county data,State abbr,County FIPS code,Reporting period,Quantity (tons),Fertilizer code,Container,Use,Grade: N,Grade: P,Grade: K
53,85.0,,IL,1.0,6.0,6195.90,428.0,2.0,0.0,0.0,0.0,60.0
54,85.0,,IL,1.0,6.0,2.00,443.0,2.0,0.0,0.0,0.0,22.0
60,85.0,,IL,3.0,6.0,4.70,428.0,2.0,0.0,0.0,0.0,60.0
79,85.0,,IL,5.0,6.0,1214.90,428.0,2.0,0.0,0.0,0.0,60.0
80,85.0,,IL,5.0,6.0,37.60,430.0,2.0,0.0,0.0,0.0,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...
29281,15.0,,SD,999.0,6.0,20.69,497.0,3.0,1.0,0.0,0.0,30.0
29283,15.0,,SD,999.0,6.0,1186.96,613.0,2.0,1.0,2.0,2.0,1.0
29284,15.0,,SD,999.0,6.0,16557.26,649.0,2.0,0.0,0.5,0.5,0.5
29285,15.0,,SD,999.0,6.0,2636.96,649.0,2.0,1.0,0.5,0.5,0.5
